<a href="https://colab.research.google.com/github/Nikhileswar-Komati/Suicide_Ideation/blob/master/XGB_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "nikhileswarkomati"
os.environ['KAGGLE_KEY'] = "001b3a30170775e55950edb6ff0c9b17"
!kaggle datasets download -d nikhileswarkomati/suicide-watch

suicide-watch.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip '/content/suicide-watch.zip'

Archive:  /content/suicide-watch.zip
replace Cleaned_Depression_Vs_Suicide.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
data = pd.read_csv('/content/Cleaned_Depression_Vs_Suicide.csv', lineterminator = '\n')
data.sample(5)

,text,class
414374,HELP ME!!I can't stop myslefw from crying! Mu ...,depression
36791,"I am a fucking messHad a plan, was planning on...",SuicideWatch
452329,worst christmas of my lifemy sister is addicte...,depression
438978,haters. haters. haters. i have no friend. more...,depression
111253,All my lies are catching up to me. Can't keep ...,depression


In [ ]:
data.shape

(609772, 2)

In [ ]:
X = data.iloc[:, 0].values
y = data.iloc[:, 1].values

le = LabelEncoder()

y = le.fit_transform(y)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size = 0.9)
print(train_X.shape, test_y.shape)

(548794,) (60978,)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

clf = Pipeline([('cv', CountVectorizer()), ('xgb', XGBClassifier())])

clf.fit(train_X, train_y)
print("------------Training Done ----------")
predictions = clf.predict(test_X)


------------Training Done ----------


In [ ]:
import numpy as np
test_acc_sklearn = np.sum(predictions == test_y) / float(len(test_y)) 


print ("Test Set Examples: ", len(test_y)) 
print ("Test Set Accuracy: ", test_acc_sklearn * 100, "%")

Test Set Examples:  60978
Test Set Accuracy:  74.82042703926007 %


In [ ]:
from joblib import dump

# dump the pipeline model
dump(clf, filename = "suicide.joblib")

['suicide.joblib']

In [ ]:
from joblib import load

# sample tweet text
text = ["I'm so depressed"]

# load the saved pipleine model
pipeline = load("/content/suicide.joblib")

# predict on the sample tweet text
pipeline.predict(text)

array([1])

In [ ]:
le.inverse_transform(pipeline.predict(text))

array(['depression'], dtype=object)

In [ ]:
import tweepy
import time
import pandas as pd
pd.set_option('display.max_colwidth', 1000)

# api key
api_key = "YfsfRUtv0Jstlvm0TLg8DrZNA"
# api secret key
api_secret_key = "Dx95SabGPVACrlQanwkajOnsfss0tWsyej8xO8rUKnf6N70Tyh"
# access token
access_token = "704330902432669696-pmTtYoAM3ywia3zAY5sWAEVkzhWUwan"
# access token secret
access_token_secret = "BSW1LmSmDZmNrDPL3KytWXgZeOTHo99Ee1vDu1FBc5EAJ"

# authorize the API Key
authentication = tweepy.OAuthHandler(api_key, api_secret_key)

# authorization to user's access token and access token secret
authentication.set_access_token(access_token, access_token_secret)

# call the api
api = tweepy.API(authentication, wait_on_rate_limit=True)

In [ ]:
def get_related_tweets(text_query):
    # list to store tweets
    tweets_list = []
    # no of tweets
    count = 50
    try:
        # Pulling individual tweets from query
        for tweet in api.search(q=text_query, count=count, lang = 'en'):
            print(tweet.text)
            # Adding to list that contains all tweets
            tweets_list.append({'created_at': tweet.created_at,
                                'tweet_id': tweet.id,
                                'tweet_text': tweet.text})
        return pd.DataFrame.from_dict(tweets_list)

    except BaseException as e:
        print('failed on_status,', str(e))
        time.sleep(3)

In [ ]:
get_related_tweets('suicide')

The most thorough follow-up of sex-reassigned people—extending over 30 years in Sweden - documents their lifelong m… https://t.co/T3IHOzjwwA
RT @Science_social9: I don't know why I feel weak asking this but does anyone have a good resource to share for black men who are seriously…
RT @hasbeemasputra: I look forward to your support in future mental health initiatives Senator @razalidris
 
You can support the mental hea…
RT @jeffgernst: An autopsy finds that the death of Keyla Martinez - who died while in police custody within hours of being detained - was a…
RT @BabySingh99: West Bengal ranks number ONE in 🌾 Vegetable Production.
BJP Sarkar should learn from  @MamataOfficial 🙏 the great CM ever.…
RT @Science_social9: I don't know why I feel weak asking this but does anyone have a good resource to share for black men who are seriously…
RT @Anonymo85632208: SUSHANT DIDN'T COMMITTED SUICIDE

@ips_nupurprasad 

FILE 302 IN SSR MURDER CASE?
RT @Science_social9: I don't know why I feel weak a

,created_at,tweet_id,tweet_text
0,2021-02-10 11:01:13,1359457388735135745,The most thorough follow-up of sex-reassigned people—extending over 30 years in Sweden - documents their lifelong m… https://t.co/T3IHOzjwwA
1,2021-02-10 11:01:11,1359457379411181576,RT @Science_social9: I don't know why I feel weak asking this but does anyone have a good resource to share for black men who are seriously…
2,2021-02-10 11:01:06,1359457359525908480,RT @hasbeemasputra: I look forward to your support in future mental health initiatives Senator @razalidris\n \nYou can support the mental hea…
3,2021-02-10 11:00:54,1359457308196167680,RT @jeffgernst: An autopsy finds that the death of Keyla Martinez - who died while in police custody within hours of being detained - was a…
4,2021-02-10 11:00:53,1359457305671081985,RT @BabySingh99: West Bengal ranks number ONE in 🌾 Vegetable Production.\nBJP Sarkar should learn from @MamataOfficial 🙏 the great CM ever.…
5,2021-02-10 11:00:53,1359457304203186181,RT @Science_social9: I don't know why I feel weak asking this but does anyone have a good resource to share for black men who are seriously…
6,2021-02-10 11:00:52,1359457301350895629,RT @Anonymo85632208: SUSHANT DIDN'T COMMITTED SUICIDE\n\n@ips_nupurprasad \n\nFILE 302 IN SSR MURDER CASE?
7,2021-02-10 11:00:51,1359457295793606661,RT @Science_social9: I don't know why I feel weak asking this but does anyone have a good resource to share for black men who are seriously…
8,2021-02-10 11:00:51,1359457295160250374,VAB's Steering Team is grateful to have recently been given the opportunity to receive Suicide Prevention Training… https://t.co/RoHrXfxfQZ
9,2021-02-10 11:00:47,1359457280169705475,@lapinleader i have the one hour version looped too😭 its way too catchy for a song about depression and suicide


In [ ]:
!python --version

Python 3.6.9


In [ ]:
!pip install xgboost

In [ ]:
import sklearn
sklearn.__version__

'0.22.2.post1'